Sometimes we want to select data based on groups and understand aggregated data on a group level. We have
seen that even though Pandas allows us to iterate over every row in a dataframe, it is geneally very slow to
do so. Fortunately Pandas has a groupby() function to speed up such task. The idea behind the groupby()
function is  that it takes some dataframe, splits it into chunks based on some key values, applies
computation on those  chunks, then combines the results back together into another dataframe. In pandas this
is refered to as the split-apply-combine pattern.

# Splitting

In [ ]:
# Let's look at an example. First, we'll bring in our pandas and numpy libraries
import pandas as pd
import numpy as np

In [ ]:
# Let's look at some US census data
#df = pd.read_csv('datasets/census.csv')
df = pd.read_csv('datasets/mpg.csv',index_col=0)

In [ ]:
df

In [ ]:
# REcall that this is masking. We use it very very frequently. 
# The most important thing to know from this lecture. 

df_audi = df[df['manufacturer']=='audi']
df_audi.head(15)

In [ ]:
# In the first example for groupby() I want to use the census date. Let's get a list of the unique states,
# then we can iterate over all the states and for each state we reduce the data frame and calculate the
# average.


In [ ]:
df['manufacturer'].unique()

In [ ]:
df['manufacturer']=='Audi'

In [ ]:
df.where(df['manufacturer']=='audi')

In [ ]:
df.where(df['manufacturer']=='audi').dropna()

In [ ]:
df.where(df['manufacturer']=='audi').dropna()['cty']

In [ ]:
np.average(df.where(df['manufacturer']=='audi').dropna()['cty'])

In [ ]:
####%%timeit -n 3

for k in df['manufacturer'].unique():
    # We'll just calculate the average using numpy for this particular state
    #avg = np.average(df.where(df['manufacturer']==k).dropna()['cty'])
    #or
    avg = np.average(df[df['manufacturer']==k]['cty'])
    # And we'll print it to the screen
    print('Car brand ' + k + 
          ' have an average consumption of ' + str(avg))

In [ ]:
df = pd.read_csv('datasets/mpg.csv',index_col=0)

for group, frame in df.groupby('manufacturer'):
    print('group\n\n',group)
    print("\n\nframe\n\n",frame)
    

In [ ]:
df.groupby('manufacturer')
#this is an iterator. We can use it just like range(). 


In [ ]:
df.head(22) #recall the origianl dataset. 

In [ ]:
###%%timeit -n 3
# For this method, we start by telling pandas we're interested in grouping by manufacturer name, this is the "split"
for k, j in df.groupby('manufacturer'):
     # k is the group name. In this example it is audi, chevrolet, etc
     # j is the data frame associated with this groups
    avg = np.average(j['cty']) ##
    #here we are getting the average cty consumption for each group
    # And print the results
    print('Car brand ' + k + 
          ' have an average consumption of ' + str(avg))
# And we don't have to worry about the combine step in this case, because all of our data transformation is
# actually printing out results.

In [ ]:
df.head

In [ ]:
# Wow, what a huge difference in speed. An improve by roughly by ten fold factors!

In [ ]:
# Let's take one more look at an example of how we might group data. In this example, I want to use a dataset
# of housing from airbnb. In this dataset there are two columns of interest, one is the cancellation_policy
# and the other is the review_scores_value.
 

In [ ]:
# So, how would I group by both of these columns? A first approach might be to promote them to a multiindex
# and just call groupby()
df = pd.read_csv('datasets/mpg.csv',index_col=0)
df=df.set_index(["manufacturer","class"])

df

In [ ]:
# When we have a multiindex we need to pass in the levels we are interested in grouping by
for i, j in df.groupby(level=(0,1)): #i for group, j for dataframe
    print(i)

In [ ]:
df.head(33)

# Applying

In [ ]:
# To this point we have applied very simple processing to our data after splitting, really just outputting
# some print statements to demonstrate how the splitting works. The pandas developers have three broad
# categories of data processing to happen during the apply step, 
# 1. Aggregation of group data, 
# 2. Transformation of  group data, and 
# 3. Filtration of group data

## Aggregation

In [34]:
# The most straight forward apply step is the aggregation of data, and uses the method agg() on the groupby
# object. Thus far we have only iterated through the groupby object, unpacking it into a label (the group
# name) and a dataframe. But with agg we can pass in a dictionary of the columns we are interested in
# aggregating along with the function we are looking to apply to aggregate.

# Let's reset the index for our  data

df = pd.read_csv('datasets/mpg.csv',index_col=0)

df=df.reset_index()

df


,index,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
0,1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...,...
229,230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
230,231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
231,232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
232,233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [35]:

# Now lets group by the manufacturer policy and find the average avearage by group
df.groupby("manufacturer").agg({"cty":np.average}) 
# every group has a frame [or df] of itself. We take the average of cty consumption of these dfs. 

,cty
manufacturer,
audi,17.611111
chevrolet,15.000000
dodge,13.135135
ford,14.000000
honda,24.444444
hyundai,18.642857
jeep,13.500000
land rover,11.500000
lincoln,11.333333


In [ ]:
df.head()

In [36]:
# np.average does not ignore "NaN". The following will ignore it. 
df.groupby("manufacturer").agg({"cty":np.nanmean})

,cty
manufacturer,
audi,17.611111
chevrolet,15.000000
dodge,13.135135
ford,14.000000
honda,24.444444
hyundai,18.642857
jeep,13.500000
land rover,11.500000
lincoln,11.333333


In [ ]:
df.head()

## Transformation

In [37]:
cols=['manufacturer','cty','hwy','trans']
df[cols]


,manufacturer,cty,hwy,trans
0,audi,18,29,auto(l5)
1,audi,21,29,manual(m5)
2,audi,20,31,manual(m6)
3,audi,21,30,auto(av)
4,audi,16,26,auto(l5)
...,...,...,...,...
229,volkswagen,19,28,auto(s6)
230,volkswagen,21,29,manual(m6)
231,volkswagen,16,26,auto(l5)
232,volkswagen,18,26,manual(m5)


In [38]:
# Transformation is different from aggregation. Where agg() returns a single value per column, so one row per
# group, tranform() returns an object that is the ***same size*** as the group. Essentially, it broadcasts the
# function you supply over the grouped dataframe, returning a new dataframe. This makes combining data later
# easy.

transform_df= df[cols].groupby('manufacturer').transform(np.nanmean)
transform_df

,cty,hwy
0,17.611111,26.444444
1,17.611111,26.444444
2,17.611111,26.444444
3,17.611111,26.444444
4,17.611111,26.444444
...,...,...
229,20.925926,29.222222
230,20.925926,29.222222
231,20.925926,29.222222
232,20.925926,29.222222


In [39]:
# So we can see that the index here is actually the same as the original dataframe. So lets just join this
# in. Before we do that, lets rename a column in the transformed version
transform_df.rename({'cty':'mean_cty'}, axis='columns', inplace=True)
transform_df.rename({'hwy':'mean_hwy'}, axis='columns', inplace=True)

df=df.merge(transform_df, left_index=True, right_index=True)
df.head()

,index,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mean_cty,mean_hwy
0,1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,17.611111,26.444444
1,2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,17.611111,26.444444
2,3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,17.611111,26.444444
3,4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,17.611111,26.444444
4,5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,17.611111,26.444444


In [40]:
# Great, we can see that our new column is in place, the mean_cty. 
# So now we could create, for instance, the difference between a given row and it's group means.

df['mean_diff']=np.absolute(df['mean_hwy']-df['mean_cty'])
df['mean_diff'].head()

0    8.833333
1    8.833333
2    8.833333
3    8.833333
4    8.833333
Name: mean_diff, dtype: float64

## Filtering

In [ ]:
# The GroupBy object has build in support for filtering groups as well. It's often that you'll want to 
# 1. group by some feature, 
# 2. then make some transformation to the groups, 
# 3. then drop certain groups as part of your cleaning routines. 
# The filter() function takes in a function which it applies to each group dataframe and
# returns either a True or a False, depending upon whether that group should be included in the results.

In [87]:
# This lambda function looks at each group of cars for a given property. 
# Now it checks for the cylnder. 
# If the mean of any group of cylinders is larger than 20, ALL OF THE GROUP  is listed.

df.groupby('class').filter(lambda x: np.nanmean(x['cty'])>20.2)

,index,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mean_cty,mean_hwy,mean_diff
90,91,ford,mustang,3.8,1999,6,manual(m5),r,18,26,r,subcompact,14.000000,19.360000,5.360000
91,92,ford,mustang,3.8,1999,6,auto(l4),r,18,25,r,subcompact,14.000000,19.360000,5.360000
92,93,ford,mustang,4.0,2008,6,manual(m5),r,17,26,r,subcompact,14.000000,19.360000,5.360000
93,94,ford,mustang,4.0,2008,6,auto(l5),r,16,24,r,subcompact,14.000000,19.360000,5.360000
94,95,ford,mustang,4.6,1999,8,auto(l4),r,15,21,r,subcompact,14.000000,19.360000,5.360000
95,96,ford,mustang,4.6,1999,8,manual(m5),r,15,22,r,subcompact,14.000000,19.360000,5.360000
96,97,ford,mustang,4.6,2008,8,manual(m5),r,15,23,r,subcompact,14.000000,19.360000,5.360000
97,98,ford,mustang,4.6,2008,8,auto(l5),r,15,22,r,subcompact,14.000000,19.360000,5.360000
98,99,ford,mustang,5.4,2008,8,manual(m6),r,14,20,p,subcompact,14.000000,19.360000,5.360000
99,100,honda,civic,1.6,1999,4,manual(m5),f,28,33,r,subcompact,24.444444,32.555556,8.111111


In [90]:
df_x = df[df['class']=="subcompact"]
df_x

,index,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mean_cty,mean_hwy,mean_diff
90,91,ford,mustang,3.8,1999,6,manual(m5),r,18,26,r,subcompact,14.000000,19.360000,5.360000
91,92,ford,mustang,3.8,1999,6,auto(l4),r,18,25,r,subcompact,14.000000,19.360000,5.360000
92,93,ford,mustang,4.0,2008,6,manual(m5),r,17,26,r,subcompact,14.000000,19.360000,5.360000
93,94,ford,mustang,4.0,2008,6,auto(l5),r,16,24,r,subcompact,14.000000,19.360000,5.360000
94,95,ford,mustang,4.6,1999,8,auto(l4),r,15,21,r,subcompact,14.000000,19.360000,5.360000
95,96,ford,mustang,4.6,1999,8,manual(m5),r,15,22,r,subcompact,14.000000,19.360000,5.360000
96,97,ford,mustang,4.6,2008,8,manual(m5),r,15,23,r,subcompact,14.000000,19.360000,5.360000
97,98,ford,mustang,4.6,2008,8,auto(l5),r,15,22,r,subcompact,14.000000,19.360000,5.360000
98,99,ford,mustang,5.4,2008,8,manual(m6),r,14,20,p,subcompact,14.000000,19.360000,5.360000
99,100,honda,civic,1.6,1999,4,manual(m5),f,28,33,r,subcompact,24.444444,32.555556,8.111111


In [ ]:
# Notice that the results are still indexed, but that any of the results which were in a group with a mean
# review score of less than or equal to 20 were not copied over.

## Applying

In [91]:
# By far the most common operation I invoke on groupby objects is the apply() function. This allows you to
# apply an arbitrary function to each group, and stitch the results back for each apply() into a single
# dataframe where the index is preserved.


df=pd.read_csv("datasets/mpg.csv")
# And lets just include some of the columns we were interested in previously
df=df[['manufacturer','cty']]
df.head(33)

,manufacturer,cty
0,audi,18
1,audi,21
2,audi,20
3,audi,21
4,audi,16
5,audi,18
6,audi,18
7,audi,18
8,audi,16
9,audi,20


In [92]:
# In previous work we wanted to find the average review score of a listing and its deviation from the group
# mean. This was a two step process, first we used transform() on the groupby object and then we had to
# broadcast to create a new column. 

# With apply() we could wrap this logic in one place
def calc_mean_cty(group):
    # group is a dataframe just of whatever we have grouped by, e.g. manufacturer, 
    #hence we can treat this as the complete dataframe
    avg=np.nanmean(group["cty"])
    #print('avg',avg)
    # now broadcast our formula and create a new column
    group["cty_diff"]=np.abs(avg-group["cty"])
    return group
calc_mean_cty(df)
# Now just apply this to the groups
df.groupby('cty').apply(calc_mean_cty).head()


,manufacturer,cty,cty_diff
0,audi,18,0.0
1,audi,21,0.0
2,audi,20,0.0
3,audi,21,0.0
4,audi,16,0.0


In [93]:
df

,manufacturer,cty,cty_diff
0,audi,18,1.141026
1,audi,21,4.141026
2,audi,20,3.141026
3,audi,21,4.141026
4,audi,16,0.858974
...,...,...,...
229,volkswagen,19,2.141026
230,volkswagen,21,4.141026
231,volkswagen,16,0.858974
232,volkswagen,18,1.141026


In [ ]:
# Using apply can be slower than using some of the specialized functions, especially agg(). But, if your
# dataframes are not huge, it's a solid general purpose approach

Groupby is a powerful and commonly used tool for data cleaning and data analysis. Once you have grouped the
data by some category you have a dataframe of just those values and you can conduct aggregated analsyis on
the segments that you are interested. The groupby() function follows a split-apply-combine approach - first
the data is split into subgroups, then you can apply some transformation, filtering, or aggregation, then
the results are combined automatically by pandas for us.